# Angular distance bias between sources and objects

- author Sylvie Dagoret-Campagne
- creation date 2024-04-05
- last update 2024-04-05
  
``Quantum Graphs``: https://tigress-web.princeton.edu/~lkelvin/pipelines/ 


Apparently isolated sources are removed from object table


 

In [ ]:
from lsst.daf.butler import Butler

import astropy.units as u
import numpy as np 
import pandas as pd
pd.set_option("display.max_columns", None)
from astropy.time import Time

import scipy.stats

import matplotlib
%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm

import lsst.afw.display.rgb as afwRgb
import lsst.afw.image as afwImage
import lsst.geom as geom
import pickle
from astropy.stats import SigmaClip

In [ ]:
plt.rcParams["figure.figsize"] = (10,6)
plt.rcParams["axes.labelsize"] = 'large'
plt.rcParams['axes.titlesize'] = 'large'
plt.rcParams['xtick.labelsize']= 'large'
plt.rcParams['ytick.labelsize']= 'large'

In [ ]:
def get_corners_radec(wcs, bbox):
    """
    Return the corners in RA,Dec in degrees given the WCS and bounding box for an image.

    Parameters
    ----------
    wcs: image WCS returned by the Butler
    bbox: bounding box returned by the Butler

    Returns
    -------
    corners_ra, corners_dec in decimal degrees
    """

    corners_x = [bbox.beginX, bbox.beginX, bbox.endX, bbox.endX]
    corners_y = [bbox.beginY, bbox.endY, bbox.endY, bbox.beginY]
    corners_ra = []
    corners_dec = []
    for i in range(4):
        radec = wcs.pixelToSky(corners_x[i], corners_y[i])
        corners_ra.append(radec.getRa().asDegrees())
        corners_dec.append(radec.getDec().asDegrees())
    
    return corners_ra, corners_dec

In [ ]:
def convert_fluxtomag(x) :
    """
    The object and source catalogs store only fluxes. There are hundreds of flux-related columns, 
    and to store them also as magnitudes would be redundant, and a waste of space.
    All flux units are nanojanskys. The AB Magnitudes Wikipedia page provides a concise resource 
    for users unfamiliar with AB magnitudes and jansky fluxes. To convert to AB magnitudes use:
    As demonstrated in Section 2.3.2, to add columns of magnitudes after retrieving columns of flux, users can do this:
    results_table['r_calibMag'] = -2.50 * numpy.log10(results_table['r_calibFlux']) + 31.4
    results_table['r_cModelMag'] = -2.50 * numpy.log10(results_table['r_cModelFlux']) + 31.4
    (from DP0 tutorial)
    """
    return -2.50 * np.log10(x) + 31.4

In [ ]:
# INSERT YOUR auxTel/LATISS collection and tract/band HERE
butlerRoot = "/repo/embargo"
# collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20231207/w_2023_49/PREOPS-4648"
#collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"
collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240311/w_2024_10/PREOPS-4985"

collectionStr = collection.replace("/", "_")
instrument = "LATISS"
skymapName = "latiss_v1"

tract = 3864
suptitle = collectionStr + f" inst = {instrument} tract = {tract}"



## Initiate butler from variables set above

In [ ]:
# Initiate butler from variables set above
butler = Butler(butlerRoot, collections=collection, instrument=instrument, skymap=skymapName)
camera = butler.get("camera", instrument=instrument)
skymap = butler.get("skyMap")
print("camera_name = {}".format(camera.getName()))
print("collection = {}".format(collection))

## load isolated_star_sources

In [ ]:
# Try to get the Schema
data_product = "isolated_star_sources"
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=collection, where= "instrument='LATISS'")
for i, ref in enumerate(datasetRefs):
    print(i,ref)
    butler_data = butler.get(ref)
    break

if not isinstance(butler_data, pd.core.frame.DataFrame):
    print(butler_data.getSchema())

In [ ]:
# Load in isolated_star_sources and trim to band of interest and select the tract
isolatedStarSourcesFull = butler.get("isolated_star_sources", tract=tract)
#isolatedStarSourcesFull = isolatedStarSourcesFull[isolatedStarSourcesFull["band"] == band]

In [ ]:
# Just to have a look at what's in the catalog:
isolatedStarSourcesFull[isolatedStarSourcesFull.index == 0]

In [ ]:
isolatedStarSourcesFull.head()

In [ ]:
isolatedStarSourcesFull.tail()

In [ ]:
isolatedStarSourcesFull.columns

In [ ]:
#print(sorted(isolatedStarSourcesFull["visit"].unique()))

In [ ]:
# Just to have a look at what's in the catalog:
isolatedStarSourcesFull[isolatedStarSourcesFull.index == 0]

In [ ]:
isolatedStarSources_g = isolatedStarSourcesFull[isolatedStarSourcesFull.band=="g"]
isolatedStarSources_r = isolatedStarSourcesFull[isolatedStarSourcesFull.band=="r"]
isolatedStarSources_i = isolatedStarSourcesFull[isolatedStarSourcesFull.band=="i"]

## Load object table

In [ ]:
#objectTable = butler.get("objectTable", tract=tract)

In [ ]:
datasetRefs = butler.registry.queryDatasets(datasetType='objectTable', collections=collection, where= "instrument='LATISS'")

In [ ]:
all_objectsTables = []

for i, ref in enumerate(datasetRefs):
    
    print("========================datasetType = objectTable ============================================")
    print("fullId..................:",ref.dataId.full)
    print("skymap...................:",ref.dataId["skymap"])
    print("tract....................:",ref.dataId["tract"])
    print("patch....................:",ref.dataId["patch"])
    print("run......................:",ref.run)

    if ref.dataId["tract"] == tract:
        df_tab=  butler.get(ref,collections=collection)
        all_objectsTables.append(df_tab) 

In [ ]:
objectTable = pd.concat(all_objectsTables)

In [ ]:
#list(objectTable.columns)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
objectTable.plot.scatter(x="coord_ra",y="coord_dec",marker="+",color='b',grid=True,ax=ax)
plt.gca().set_aspect('equal')
#plt.grid()

In [ ]:
#objectTable[objectTable.detect_isPrimary].refSizeExtendedness.hist(bins=50)

In [ ]:
selection = (objectTable.refSizeExtendedness<0.1) & (objectTable.detect_isPrimary) & (objectTable.detect_isIsolated)

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
objectTable[selection].plot.scatter(x="coord_ra",y="coord_dec",marker="+",color='k',ax=ax)
isolatedStarSources_g.plot.scatter(x="ra",y="dec",marker="+",color="g",ax=ax,label="sources g")
plt.gca().set_aspect('equal')
plt.grid()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
objectTable[selection].plot.scatter(x="coord_ra",y="coord_dec",marker="+",color='k',ax=ax)
isolatedStarSources_r.plot.scatter(x="ra",y="dec",marker="+",color="r",ax=ax,label="sources r")
plt.gca().set_aspect('equal')
plt.grid()

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(8,8))
objectTable[selection].plot.scatter(x="coord_ra",y="coord_dec",marker="+",color='k',ax=ax)
isolatedStarSources_i.plot.scatter(x="ra",y="dec",marker="+",color="orange",ax=ax,label="sources i")
plt.gca().set_aspect('equal')
plt.grid()

In [ ]:
#objectTable[objectTable.refSizeExtendedness>0.9].plot.scatter(x="coord_ra",y="coord_dec",marker="+")

In [ ]:
isolatedObjects = objectTable[selection]

In [ ]:
isolatedObjects["g_psfMag"] = isolatedObjects["g_psfFlux"].map(convert_fluxtomag)
isolatedObjects["r_psfMag"] = isolatedObjects["r_psfFlux"].map(convert_fluxtomag)
isolatedObjects["i_psfMag"] = isolatedObjects["i_psfFlux"].map(convert_fluxtomag)
#isolatedObjects["z_psfMag"] = isolatedObjects["z_psfFlux"].apply(convert_fluxtomag)
#isolatedObjects["y_psfMag"] = isolatedObjects["y_psfFlux"].apply(convert_fluxtomag)

In [ ]:
fig,axs = plt.subplots(1,3,figsize=(18,4))
ax1,ax2,ax3 = axs.flatten()
isolatedObjects["g_psfMag"].hist(bins=50,facecolor="g",ax=ax1,legend="g")
isolatedObjects["r_psfMag"].hist(bins=50,facecolor="r",ax=ax2,legend="r")
isolatedObjects["i_psfMag"].hist(bins=50,facecolor="orange",ax=ax3,legend="i")

In [ ]:
isolatedObjects["g_r"] = isolatedObjects["g_psfMag"] - isolatedObjects["r_psfMag"]
isolatedObjects["r_i"] = isolatedObjects["r_psfMag"] - isolatedObjects["i_psfMag"]

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(6,6))

isolatedObjects.plot.scatter(y="r_i", x="g_r",marker = "+" ,color="b",ax=ax)
ax.set_xlim(0,4)
ax.set_ylim(-1,3)
ax.grid()
ax.set_title("Color-Color plot on objectTable")